In [1]:
import pandas as pd
import numpy as np
from gcmap import GCMapper, Gradient
from PIL import Image, ImageFilter

In [2]:
ROUTE_COLS = ('airline_name', 'airline_id', 'source_code', 'source_id', 'dest_code', 'dest_id', 'codeshare', 'stops', 'equiptment')
AIRPORT_COLS = ('airport_id', 'airport_name', 'city', 'country', 'iata', 'icao', 'latitude', 'longitude', 'altitude', 'timezone', 'dst', 'tz database', 'type', 'source')

routes = pd.read_csv('data/routes.dat', header=None, names=ROUTE_COLS, na_values=['\\N'])
airports = pd.read_csv('data/airports.dat', header=None, names=AIRPORT_COLS)

In [3]:
routes.drop(['codeshare', 'stops', 'equiptment'], axis=1)

,airline_name,airline_id,source_code,source_id,dest_code,dest_id
0,2B,410.0,AER,2965.0,KZN,2990.0
1,2B,410.0,ASF,2966.0,KZN,2990.0
2,2B,410.0,ASF,2966.0,MRV,2962.0
3,2B,410.0,CEK,2968.0,KZN,2990.0
4,2B,410.0,CEK,2968.0,OVB,4078.0
...,...,...,...,...,...,...
67658,ZL,4178.0,WYA,6334.0,ADL,3341.0
67659,ZM,19016.0,DME,4029.0,FRU,2912.0
67660,ZM,19016.0,FRU,2912.0,DME,4029.0
67661,ZM,19016.0,FRU,2912.0,OSS,2913.0


In [4]:
airports.drop(['icao', 'timezone', 'dst', 'tz database', 'source'], axis=1)

,airport_id,airport_name,city,country,iata,latitude,longitude,altitude,type
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,-6.081690,145.391998,5282,airport
1,2,Madang Airport,Madang,Papua New Guinea,MAG,-5.207080,145.789001,20,airport
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,-5.826790,144.296005,5388,airport
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,-6.569803,146.725977,239,airport
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,-9.443380,147.220001,146,airport
...,...,...,...,...,...,...,...,...,...
7693,14106,Rogachyovo Air Base,Belaya,Russia,\N,71.616699,52.478298,272,airport
7694,14107,Ulan-Ude East Airport,Ulan Ude,Russia,\N,51.849998,107.737999,1670,airport
7695,14108,Krechevitsy Air Base,Novgorod,Russia,\N,58.625000,31.385000,85,airport
7696,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,-27.261200,-70.779198,670,airport


In [5]:
airport_pairs = routes.groupby(('source_id', 'dest_id')).size()
airport_pairs = airport_pairs.reset_index()
airport_pairs.columns = ('source_id', 'dest_id', 'cnt')

/home/jan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


In [6]:
airport_pairs

,source_id,dest_id,cnt
0,1.0,2.0,1
1,1.0,3.0,1
2,1.0,4.0,1
3,1.0,5.0,2
4,2.0,1.0,1
...,...,...,...
37269,11473.0,3379.0,1
37270,11473.0,6341.0,2
37271,11473.0,6432.0,2
37272,11498.0,3399.0,1


In [7]:
airport_pairs = airport_pairs.merge(airports, left_on='source_id', right_on='airport_id') \
                            .merge(airports, left_on='dest_id', right_on='airport_id', suffixes=('_source', '_dest'))

In [8]:
g = Gradient([(   0,   204,   255,   131),
              ( 0.4,    51,   255,    51),
              (   1,     0,     0,     0)])


gcm = GCMapper(cols=g, bgcol=(255,255,255))
gcm.set_data(airport_pairs.longitude_source, airport_pairs.latitude_source, airport_pairs.longitude_dest, airport_pairs.latitude_dest, airport_pairs.cnt)
img = gcm.draw()
img.save('test.png')

original = Image.open("test.png")
original.show()

/home/jan/anaconda3/lib/python3.7/site-packages/gcmap/gcmap.py:86: RuntimeWarning: divide by zero encountered in true_divide
  self.weight = np.array(count) / dist


# Creating Inter-country Routes (in .csv)

In [9]:
airport_pairs

,source_id,dest_id,cnt,airport_id_source,airport_name_source,city_source,country_source,iata_source,icao_source,latitude_source,...,iata_dest,icao_dest,latitude_dest,longitude_dest,altitude_dest,timezone_dest,dst_dest,tz database_dest,type_dest,source_dest
0,1.0,2.0,1,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,...,MAG,AYMD,-5.20708,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
1,3.0,2.0,1,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,...,MAG,AYMD,-5.20708,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,4.0,2.0,1,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,...,MAG,AYMD,-5.20708,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
3,5.0,2.0,1,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,...,MAG,AYMD,-5.20708,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
4,6.0,2.0,2,6,Wewak International Airport,Wewak,Papua New Guinea,WWK,AYWK,-3.583830,...,MAG,AYMD,-5.20708,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36902,8243.0,7368.0,1,8243,São Félix do Araguaia Airport,Sao Felix do Araguaia,Brazil,SXO,SWFX,-11.632400,...,GRP,SWGI,-11.73960,-49.132198,1148,-3,S,America/Fortaleza,airport,OurAirports
36903,9229.0,3030.0,1,9229,Sigiriya Air Force Base,Sigiriya,Sri Lanka,GIU,VCCS,7.956670,...,TRR,VCCT,8.53851,81.181900,6,5.5,U,Asia/Colombo,airport,OurAirports
36904,9739.0,9744.0,1,9739,Napakiak Airport,Napakiak,United States,WNA,PANA,60.690300,...,PKA,PAPK,60.70290,-161.778000,24,-9,A,America/Anchorage,airport,OurAirports
36905,11229.0,1033.0,1,11229,Beni Airport,Beni,Congo (Kinshasa),BNC,FZNP,0.575000,...,BUX,FZKA,1.56572,30.220800,4045,2,U,Africa/Lubumbashi,airport,OurAirports


In [28]:
country_routes = airport_pairs.groupby(('country_source', 'country_dest'))
country_routes = country_routes.aggregate(np.sum)

/home/jan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


In [29]:
country_routes

source_id  dest_id  cnt  airport_id_source  \
country_source country_dest                                               
Afghanistan    Afghanistan     12302.0  12302.0   14              12302   
               Azerbaijan       2050.0   2922.0    1               2050   
               India            6149.0   9279.0    7               6149   
               Iran            10252.0  21653.0    5              10252   
               Kuwait           2050.0   2176.0    1               2050   
...                                ...      ...  ...                ...   
Zimbabwe       Namibia          2009.0   8210.0    2               2009   
               South Africa     4015.0   3238.0    9               4015   
               Tanzania         1005.0   1177.0    1               1005   
               Zambia           1005.0    907.0   11               1005   
               Zimbabwe         6020.0   6020.0    6               6020   

                             latitude_source  longitude_source  \
country_source country_dest                                      
Afghanistan    Afghanistan        206.120398        402.922710   
               Azerbaijan          34.565899         69.212303   
               India              100.281698        197.288402   
               Iran               171.554499        333.710407   
               Kuwait              34.565899         69.212303   
...                                      ...               ...   
Zimbabwe       Namibia            -36.027701         56.931801   
               South Africa       -73.976903        116.642502   
               Tanzania           -17.931801         31.092800   
               Zambia             -17.931801         31.092800   
               Zimbabwe          -112.090203        171.099403   

                             altitude_source  airport_id_dest  latitude_dest  \
country_source country_dest                                                    
Afghanistan    Afghanistan             25458            12302     206.120398   
               Azerbaijan               5877             2922      40.467499   
               India                   12420             9279      85.699500   
               Iran                    19581            21653     179.537796   
               Kuwait                   5877             2176      29.226601   
...                                      ...              ...            ...   
Zimbabwe       Namibia                  8377             8210     -44.959800   
               South Africa            17623             3238    -108.032044   
               Tanzania                 4887             1177      -6.878110   
               Zambia                   4887              907     -15.330800   
               Zimbabwe                25472             6020    -112.090203   

                             longitude_dest  altitude_dest  
country_source country_dest                                 
Afghanistan    Afghanistan       402.922710          25458  
               Azerbaijan         50.046700             10  
               India             231.309312           2331  
               Iran              281.227394          16399  
               Kuwait             47.968899            206  
...                                     ...            ...  
Zimbabwe       Namibia            34.941800          11280  
               South Africa      115.857722          16969  
               Tanzania           39.202599            182  
               Zambia             28.452600           3779  
               Zimbabwe          171.099403          25472  

[4697 rows x 11 columns]

In [30]:
country_routes.loc['China',:]

,source_id,dest_id,cnt,airport_id_source,latitude_source,longitude_source,altitude_source,airport_id_dest,latitude_dest,longitude_dest,altitude_dest
country_dest,,,,,,,,,,,
Algeria,3364.0,210.0,1,3364,40.080101,116.584999,116,210,36.691002,3.215410,82
Angola,3364.0,951.0,1,3364,40.080101,116.584999,116,951,-8.858370,13.231200,243
Australia,33832.0,33493.0,17,33832,287.976206,1142.842010,3381,33493,-342.074897,1459.498023,1487
Austria,3364.0,1613.0,2,3364,40.080101,116.584999,116,1613,48.110298,16.569700,600
Azerbaijan,6763.0,5844.0,2,6763,83.987202,204.059196,2241,5844,80.934998,100.093399,20
...,...,...,...,...,...,...,...,...,...,...,...
United Arab Emirates,23687.0,15289.0,14,23687,221.519848,796.955168,8836,15289,174.310202,385.410896,512
United Kingdom,16899.0,2530.0,11,16899,165.274504,572.221001,1920,2530,257.030502,-2.038042,534
United States,64258.0,68796.0,58,64258,673.340612,2253.588997,1314,68796,714.580245,-1971.649251,4398


In [33]:
country_routes = country_routes.reset_index()
ext_country_routes = country_routes[country_routes['country_source'] != country_routes['country_dest']]
ext_country_routes

,level_0,index,country_source,country_dest,source_id,dest_id,cnt,airport_id_source,latitude_source,longitude_source,altitude_source,airport_id_dest,latitude_dest,longitude_dest,altitude_dest
0,0,1,Afghanistan,Azerbaijan,2050.0,2922.0,1,2050,34.565899,69.212303,5877,2922,40.467499,50.046700,10
1,1,2,Afghanistan,India,6149.0,9279.0,7,6149,100.281698,197.288402,12420,9279,85.699500,231.309312,2331
2,2,3,Afghanistan,Iran,10252.0,21653.0,5,10252,171.554499,333.710407,19581,21653,179.537796,281.227394,16399
3,3,4,Afghanistan,Kuwait,2050.0,2176.0,1,2050,34.565899,69.212303,5877,2176,29.226601,47.968899,206
4,4,5,Afghanistan,Pakistan,6148.0,6665.0,4,6148,103.341797,200.652904,14960,6665,101.227298,217.712997,4494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4553,4553,4691,Zimbabwe,Malawi,1005.0,1016.0,1,1005,-17.931801,31.092800,4887,1016,-13.789400,33.780998,4035
4554,4554,4692,Zimbabwe,Namibia,2009.0,8210.0,2,2009,-36.027701,56.931801,8377,8210,-44.959800,34.941800,11280
4555,4555,4693,Zimbabwe,South Africa,4015.0,3238.0,9,4015,-73.976903,116.642502,17623,3238,-108.032044,115.857722,16969
4556,4556,4694,Zimbabwe,Tanzania,1005.0,1177.0,1,1005,-17.931801,31.092800,4887,1177,-6.878110,39.202599,182


In [34]:
ext_country_routes.to_csv (r'/home/jan/Code/MasterThesis/data/country_flight_country.csv', index = False, header=True)